In [1]:
import pandas as pd
import numpy as np
import re
import time
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore")

In [1]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']

invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

oversample = True

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)

In [ ]:
tic = time.time()
tmp = data.groupby(['问题ID'])['用户ID'].apply(lambda x:','.join(x)).reset_index()
tmp.columns = ['问题ID','问题_用户IDs']
data = data.merge(tmp, on='问题ID', how='left')
    
tmp = data.groupby(['用户ID'])['问题ID'].apply(lambda x:','.join(x)).reset_index()
tmp.columns = ['用户ID','用户_问题IDs']
data = data.merge(tmp, on='用户ID', how='left')
print('Used time: %d' % int(toc-tic))

In [ ]:
tic = time.time()
X_1 = TfidfVectorizer(token_pattern='[A-Z0-9]+',binary=True).fit_transform(data['用户_问题IDs'])
ques_svd = TruncatedSVD(n_components=10,algorithm='randomized',n_iter=30,random_state=2019).fit_transform(X_1)
ques_svd_df = pd.DataFrame(ques_svd, columns=['用户_问题IDs_svd_{}'.format(i) for i in range(1,11)])
ques_svd_df[:len(invite_info)+len(invite_info_evaluate_B)].to_pickle(os.path.join(FEAT_PATH, 'user_quesIDs_svd_.pickle'))
    
X_1 = TfidfVectorizer(token_pattern='[A-Z0-9]+',binary=True).fit_transform(data['问题_用户IDs'])
ques_svd = TruncatedSVD(n_components=10,algorithm='randomized',n_iter=30,random_state=2019).fit_transform(X_1)
ques_svd_df = pd.DataFrame(ques_svd, columns=['问题_用户IDs_svd_{}'.format(i) for i in range(1,11)])
ques_svd_df[:len(invite_info)+len(invite_info_evaluate_B)].to_pickle(os.path.join(FEAT_PATH, 'ques_userIDs_svd.pickle'))

toc = time.time()
print('Used time: %d' % int(toc-tic))